In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
import geopandas as gpd
import topojson as tp
from shapely import geometry

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [5]:
url='https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/historial_comunas.csv'

In [6]:
contagiados_comunal=pd.read_csv(url)
contagiados_comunal['Fecha']=pd.to_datetime(contagiados_comunal['Fecha'],errors='coerce').dt.to_pydatetime()

In [7]:
for k in ['@ivanMSC']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema @ivanMSC exists, will not be created


In [8]:
name='contagios_comunal@ivanMSC'
schema='@ivanMSC'
print("creating table "+name+' ,schema: '+schema)
contagiados_comunal.to_sql(name, schema=schema,con=cnx,if_exists='replace')
print("saving table"+path+name+timestamp+'.csv in cache')
contagiados_comunal.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

creating table contagios_comunal@ivanMSC ,schema: @ivanMSC
saving table/data/ETLcache/contagios_comunal@ivanMSC_13092020_185800.csv in cache


In [9]:
contagiados_comunal.Region.unique()

array(['AP', 'TA', 'AN', 'AT', 'CO', 'VS', 'RM', 'LI', 'ML', 'NB', 'BI',
       'AR', 'LR', 'LL', 'AI', 'MA'], dtype=object)

In [10]:
contagiados_comunal['Region']=contagiados_comunal['Region'].replace({'AP':'Arica y Parinacota', 'TA':'Tarapacá', 'AN':'Antofagasta', 'AT':'Atacama', 
                                       'CO':'Coquimbo', 'VS':'Valparaíso', 'RM':'Región Metropolitana', 'LI':"OHiggins", 'ML':'Maule', 'NB':'Ñuble', 
                                       'BI':'Bío-Bío',
       'AR':'Araucanía', 'LR':'Los Ríos', 'LL':'Los Lagos', 'AI':'Aysén', 'MA':'Magallanes'})

In [23]:
name='contagios_comunal'
schema='@ivanMSC'
contagiados_comunal.to_sql(name=name,schema=schema, con=cnx)

ProgrammingError: (psycopg2.errors.UndefinedObject) type "geometry" does not exist
LINE 4:  geometry geometry(GEOMETRY,3857), 
                  ^

[SQL: 
CREATE TABLE "@ivanMSC".contagios_comunal (
	cod_comuna BIGINT, 
	geometry geometry(GEOMETRY,3857), 
	"Fecha" TIMESTAMP WITHOUT TIME ZONE, 
	"CUT" FLOAT(53), 
	"Region" TEXT, 
	"Comuna" TEXT, 
	"Confirmados Acumulados" FLOAT(53), 
	"Nuevos Confirmados" FLOAT(53)
)

]
(Background on this error at: http://sqlalche.me/e/13/f405)